In [15]:
import numpy as np
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from d2l import torch as d2l
from more.datasets import ModulationDataSets
from more.network import MoreNet

In [17]:
train_path = r'E:\train.h5'
test_path = r'E:\test.h5'
train_dataset = ModulationDataSets(train_path)
test_dataset = ModulationDataSets(test_path)
label_name = train_dataset.get_labels_name();

In [19]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)

In [24]:
net = MoreNet()
X = torch.rand(size=(1, 1, 196, 64), dtype=torch.float32)
net(X,True)

C:\Users\ZGYwi\AppData\Local\Temp\ipykernel_10384\551632285.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return x.detach(),F.softmax(y)


(tensor([[-0.0012, -0.0070]]),
 tensor([[0.0860, 0.1036, 0.0753, 0.0989, 0.1995, 0.1398, 0.2043, 0.0925]],
        grad_fn=<SoftmaxBackward0>))

In [10]:
net.load_state_dict(torch.load('basic-8.parm'))

RuntimeError: Error(s) in loading state_dict for MoreNet:
	Missing key(s) in state_dict: "fc3.weight", "fc3.bias". 
	Unexpected key(s) in state_dict: "fc8.weight", "fc8.bias". 
	size mismatch for fc2.weight: copying a param with shape torch.Size([2, 8]) from checkpoint, the shape in current model is torch.Size([2, 50]).

In [ ]:
torch.save(net.state_dict(),"basic-8.parm")

In [25]:

import matplotlib.pyplot as plt
import os.path as osp
import os
def plot_features(features, labels, num_classes, epoch, prefix):
    """Plot features on 2D plane.

    Args:
        features: (num_instances, num_features).
        labels: (num_instances).
    """
    colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    for label_idx in range(num_classes):
        plt.scatter(
            features[labels==label_idx, 0],
            features[labels==label_idx, 1],
            c=colors[label_idx],
            s=1,
        )
    plt.legend(label_name, loc='upper right')
    dirname = osp.join(r"E:\pic", prefix)
    if not osp.exists(dirname):
        os.mkdir(dirname)
    save_name = osp.join(dirname, 'epoch_' + str(epoch+1) + '.png')
    plt.savefig(save_name, bbox_inches='tight')
    plt.close()
def train_ch6(net, loss, train_iter, test_iter, num_epochs, lr, lambd, device):
    """Train a model with a GPU (defined in Chapter 6). """

    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.5,
                                weight_decay=lambd)

    # animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
    #                         legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    features,feature_label=[],[]
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            feat,y_hat = net(X,True)

            features.append(feat.data.cpu().numpy())
            feature_label.append(y.data.cpu().numpy())

            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            # if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
            #     animator.add(epoch + (i + 1) / num_batches,
            #                  (train_l, train_acc, None))
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        # animator.add(epoch + 1, (None, None, test_acc))

        all_features = np.concatenate(features, 0)
        all_labels = np.concatenate(feature_label, 0)
        plot_features(all_features,all_labels,8,epoch,prefix='more')


    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')
def init_weights(net: nn.Module):
    def init(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    net.apply(init)

In [26]:
init_weights(net)

In [27]:
loss= nn.CrossEntropyLoss()

In [28]:
n_epochs = 10
train_ch6(net, loss, train_loader, test_loader, n_epochs, 0.1, 0, d2l.try_gpu())

training on cuda:0


C:\Users\ZGYwi\AppData\Local\Temp\ipykernel_10384\551632285.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return x.detach(),F.softmax(y)
C:\Users\ZGYwi\AppData\Local\Temp\ipykernel_10384\551632285.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(y)


KeyboardInterrupt: 

In [ ]:
n_epochs =20
train_ch6(net, loss, train_loader, test_loader, n_epochs, 0.01, 0, d2l. v())

In [ ]:
torch.save(net.state_dict(),"basic-8.parm")

In [ ]:
import matplotlib.pyplot as plt
import os.path as osp
import os
def plot_features(features, labels, num_classes, epoch, prefix):
    """Plot features on 2D plane.
    Args:
        features: (num_instances, num_features).
        labels: (num_instances).
    """
    colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    for label_idx in range(num_classes):
        plt.scatter(
            features[labels==label_idx, 0],
            features[labels==label_idx, 1],
            c=colors[label_idx],
            s=1,
        )
    plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], loc='upper right')
    dirname = osp.join(r"E:\pic", prefix)
    if not osp.exists(dirname):
        os.mkdir(dirname)
    save_name = osp.join(dirname, 'epoch_' + str(epoch+1) + '.png')
    plt.savefig(save_name, bbox_inches='tight')
    plt.close()